# Fine-tuning T5 for Summarization

Example with t5-small using Hugging Face Trainer.

In [ ]:
# !pip install transformers datasets sentencepiece

## Load model and tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained('t5-small')
model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')
print('T5 loaded')

## Prepare tiny dataset

In [ ]:
from datasets import Dataset
data = {'text':['Deep learning is a subset of machine learning that uses neural networks.'], 'summary':['Deep learning uses neural networks.']}
ds = Dataset.from_dict(data)

def preprocess(batch):
    inputs = tokenizer(['summarize: '+t for t in batch['text']], truncation=True, padding='max_length', max_length=128)
    labels = tokenizer(batch['summary'], truncation=True, padding='max_length', max_length=32)
    inputs['labels'] = labels['input_ids']
    return inputs

ds = ds.map(preprocess, batched=True)
ds.set_format(type='torch', columns=['input_ids','attention_mask','labels'])
print(ds[0])

## Training (Trainer example)

In [ ]:
from transformers import Trainer, TrainingArguments
args = TrainingArguments(output_dir='./t5_out', per_device_train_batch_size=4, num_train_epochs=3, logging_steps=10)
trainer = Trainer(model=model, args=args, train_dataset=ds)
# trainer.train()  # Uncomment to run training
print('Trainer ready. Uncomment trainer.train() to start training.')